# Confronto Insertion-sort e heap sort

In questo notebook verranno analizzati e confrontati i due algoritmi di ordinamento *insertion-sort* e *heap-sort*. 

## Studio prestazioni attese
Prima di procedere con l'implementazione degli algoritmi è opportuno fare delle assunzioni sulle prestazioni che ci aspettiamo.

### Insertion sort

L'insertion sort è un algoritmo di ordinamento **stabile** che ordina **sul posto** ossia opera sull'array da ordinare senza aver bisogno di memoria aggiuntiva. 

La logica secondo cui agisce questo algoritmo si potrebbe paragonare al modo in cui viene solitamente ordinato un mazzo di carte: l'insertion sort opera infatti ordinando gradualmente un sottoarray sinistro dell'input fornito, incrementando di 1 la grandezza del sottoarray ad ogni iterazione fino a quando l'intero array non è ordinato.
#### Costo Insertion sort

Possiamo quindi dedurre che nel **caso migliore**, ossia quando l'input è un array ordinato, l'insertion sort analizzera ogni elemento dell'array fino alla fine senza operare alcun cambiamento e dunque avremo un **costo lineare $O(n)$**. 

Nel **caso peggiore** ossia con in input un array ordinato al contrario l'array avrà un **costo quadratico $O(n^2)$**. 

Anche nel **caso medio** il costo sarà **quadratico $O(n^2)$**.
#### Esempio applicazione insertion sort

<img src = "insertion.png" alt = "esempio insertion sort" width = 50%/>

### Heap sort

L'heap sort è un algoritmo di ordinamento tipicamente **non stabile** basato su confronti che ordina **sul posto**. 

L'heap sort opera trasformando un array in un max-heap, un albero binario rappresentato utilizzando un array dove il massimo valore si trova alla radice e scendendo dalla radice verso le foglie i valori diminuiscono, ossia vale la proprietà: 
$$A[parent(i)] \ge A[i]$$

La rappresentazione tramite array di un albero binario sfrutta le seguenti proprietà: 
- Il primo elemento dell'array è la radice dell'albero
- Il padre del nodo $i$ è il nodo $\lfloor i/2 \rfloor$
- Il figlio sinistro del nodo  $i$ è il nodo $2i$
- Il figlio destro del nodo  $i$ è il nodo $2i + 1$
#### Esempio rapresentazione di un albero binario tramite array

<img src = "Array-Complete-Tree.png" alt = "esempio insertion sort" width = 50%/>

#### Costo Insertion sort

Lo heap sort è composto da due parti, *Max-Heapify* e *Build-Max-Heap*. Max-Heapify ha costo $O(lgn)$, mentre Build-max-heap effettua $n/2$ chiamate a Max-heapify, dunque abbiamo un costo complessivo di $O(n lgn)$.

## Test effettuati
Gli algoritmi saranno testati secondo tre modalità: 
- Con valori randomizzati
- Con valori crescenti 
- Con valori decrescenti

I risultati dei test saranno rappresentati tramite la libreria *matplotlib*. 

## Codice implementato

Import delle librerie necessarie

In caso si verifichi un *ModuleNotFoundError* per matplotlib eseguire la riga sottostante: 

In [3]:
import sys  
!{sys.executable} -m pip install --user matplotlib

  Using cached matplotlib-3.7.0-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (9.2 MB)
  Using cached numpy-1.24.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached fonttools-4.38.0-py3-none-any.whl (965 kB)
  Using cached contourpy-1.0.7-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
  Using cached kiwisolver-1.4.4-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.2 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)


In [5]:
import random
from timeit import default_timer as timer
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

## Codice insertion sort

In [6]:
def insertion_sort(A):
    for j in range(1, len(A)):
        key = A[j]
        i = j - 1
        while i >= 0 and A[i] > key:
            A[i + 1] = A[i]
            i -= 1
        A[i + 1] = key

## Codice heapSort

In [7]:
def maxHeapify(arr, heapSize, i):
    l = left(i)  # imposta l'indice del figlio sinistro
    r = right(i)  # imposta l'indice del figlio destro

    if l < heapSize and arr[i] < arr[l]:  # controlliamo di non aver sforato l'array e confrontiamo i e left
        maxVal = l
    else:
        maxVal = i

    if r < heapSize and arr[r] > arr[maxVal]:
        maxVal = r
    # se i non è il valore più grande allora dobbiamo scambiarlo con il maggiore per mantenere la proprietà
    # di max heap
    if maxVal != i:
        arr[i], arr[maxVal] = arr[maxVal], arr[i]  # scambia a[i] con a[max]
        maxHeapify(arr, heapSize, maxVal)


In [8]:
def heapSort(arr):
    buildMaxHeap(arr)
    fixPositions(arr, len(arr))

In [9]:
def fixPositions(arr, heapSize):
    for i in range(heapSize - 1, 0, -1):  # scorre l'array al contrario
        arr[i], arr[0] = arr[0], arr[i]  # scambia zero con il valore massimo del sottoarray che va da zero ad i
        # stiamo chiamando maxheapify su un array di grandezza i, stiamo ponendo in posizione zero il valore
        # più grande del sottoarray
        maxHeapify(arr, i, 0)


def buildMaxHeap(arr):
    heapSize = len(arr)
    for i in range(heapSize // 2 - 1, -1, -1):
        maxHeapify(arr, heapSize, i)


def left(i):
    return 2 * i + 1


def right(i):
    return 2 * i + 2

## Classe Tester

La classe tester esegue i test secondo le tre modalità descritte in precedenza e restituisce il risultato

In [18]:
class Tester:
    def __init__(self, n, testedAlgorithms):
        self.sequentialArrays = None
        self.randomArrays = None
        self.n = n
        self.testedAlgorithms = testedAlgorithms

    def generateSequentialValues(self):
        self.sequentialArrays = []
        self.initializeMultidimensionalArray(self.sequentialArrays, self.n)
        self.sequentialArrays[0] = [0]
        for i in range(1, self.n):
            self.sequentialArrays[i] = self.sequentialArrays[i - 1].copy()
            self.sequentialArrays[i].append(i)

    def generateRandomValues(self, lowerB, upperB):
        self.randomArrays = []
        self.initializeMultidimensionalArray(self.randomArrays, self.n)
        for i in range(0, self.n):
            for j in range(0, i + 1):
                self.randomArrays[i].append(random.randrange(lowerB, upperB))

    def initializeMultidimensionalArray(self, array, end):
        for i in range(0, end):
            array.append([])

    def runRandomizedTest(self, lowerB, upperB):
        self.generateRandomValues(lowerB, upperB)
        return self.runTest(self.randomArrays)

    def runSequentialTests(self):
        self.generateSequentialValues()
        return self.runTest(self.sequentialArrays)

    def runReversedSequentialTests(self):
        self.generateSequentialValues()
        data = []
        for i in range(0, len(self.sequentialArrays)):
            data.append(list(reversed(self.sequentialArrays[i])))
        return self.runTest(data)

    def runTest(self, array):
        results = []
        self.initializeMultidimensionalArray(results, self.testedAlgorithms)
        data = []
        for i in range(0, self.testedAlgorithms):
            data.append(array.copy())

        for j in range(0, self.n):
            index = 0
            previous = results[index][j - 1][1] if j != 0 else None  # imposta il valore del precedente
            results[index].append((j, self._testInsertion(data[index][j], previous)))
            index += 1
            previous = results[index][j - 1][1] if j != 0 else None
            results[index].append((j, self._testHeapSort(data[index][j], previous)))
        return results

    def _testInsertion(self, data, previous):
        start = timer()
        insertionSort(data)
        end = timer()
        return end - start if previous is None else end - start + previous

    def _testHeapSort(self, data, previous):
        start = timer()
        heapSort(data)
        end = timer()
        return end - start if previous is None else end - start + previous



## Classe Plotter

La classe plotter permette di realizzare i plot dei risultati forniti in input

In [19]:
class Plotter:
    def plotGraph(self, x, toPlot, graphLabels, colors, graphTitle, axisLabels):
        for i in range(0, len(toPlot)):
            plt.plot(x, toPlot[i], colors[i])
        plt.legend(graphLabels, title=graphTitle)
        plt.xlabel = axisLabels[0]
        plt.ylabel = (axisLabels[1])
        plt.show()


Il metodo *getYAxis* converte i risultati forniti da Tester in un array interpretabile dalla classe Plotter. 

In [20]:
def getYAxis (result, index):
    y = []
    for val in result[index]:
        y.append(val[1])
    return y

In [21]:
n = 100
numTested = 2
lowerB = 0
upperB = 10

t = Tester(n, numTested)
r1 = t.runRandomizedTest(lowerB, upperB)
r2 = t.runSequentialTests()
r3 = t.runReversedSequentialTests()
p = Plotter()

y1 = getYAxis(r1,0)
y2 = getYAxis(r2,0)
y3 = getYAxis(r3,0)
y4 = getYAxis(r1, 1)
y5 = getYAxis(r2, 1)
y6 = getYAxis(r3, 1)

x = [i for i in range (0, n)]

p.plotGraph(x,[y1, y2, y3],["randomized", "sequential", "reversed"],["-r", "-m", "-g"],"Sequential sort",["nNodi", "tempo"])

NameError: name 'insertionSort' is not defined